In [5]:
import pandas as pd
rna_data = pd.read_csv("gene_data/rna_common_complete.csv")
rna_data = rna_data.sort_values(by=['sn','period']).reset_index(drop=True)

In [6]:
X_og_shape = rna_data.drop(['sn','group','caarms_status','period'],axis=1).values
X_reshaped = X_og_shape.reshape(len(set(rna_data['sn'])), 3, X_og_shape.shape[1])
labels_group = rna_data[rna_data['period'] == 24]['group'].values
labels = [0 if i == 'C' else 1 for i in labels_group]

In [7]:
import torch
from neucube.utils import SNR
from neucube.utils import interpolate
from neucube.encoder import Delta

ratios = SNR(X_reshaped[:,0,:], labels)
top_idx = torch.argsort(ratios, descending=True)[0:20]
X_reshaped_topidx = X_reshaped[:,:,top_idx]
interpolated_X = interpolate(X_reshaped_topidx, num_points=104)

encoder = Delta(threshold=0.008)
X = encoder.encode_dataset(interpolated_X)
y = torch.tensor(labels)

In [8]:
from neucube.sampler import TemporalBinning
from neucube.utils import SeparationIndex
def objective_function(res_, X_stimuli, labels, params):
    a, b, c, d = params
    res_.set_exc_parms(a=a, b=b, c=c, d=d)
    out_spikes = res_.simulate(X_stimuli, mem_thr=30, train=False, verbose=False)
    sampler = TemporalBinning(bin_size=10)
    state_vec = sampler.sample(out_spikes)
    # state_vec_SNR = SNR(state_vec,y)
    # sv_top_feat = torch.argsort(state_vec_SNR, descending=True)[0:100]
    # state_vec_top = state_vec[:,sv_top_feat]
    return SeparationIndex(state_vec, labels).item()

In [9]:
from neucube import IzhReservoir
izh_res = IzhReservoir(inputs=X.shape[2], c=0.7, l=0.18, input_conn_prob=0.85)
izh_res.set_exc_parms(a=0.06, b=0.55, c=-55, d=3)
izh_res.set_inh_parms(a=0.01, b=0.2, c=-65, d=8)

In [10]:
import nevergrad as ng
import numpy as np
from functools import partial

parametrization = ng.p.Tuple(
    ng.p.Scalar(init=izh_res.a.cpu()[0]),
    ng.p.Scalar(init=izh_res.b.cpu()[0]), 
    ng.p.TransitionChoice(list(range(-65,-46))),
    ng.p.TransitionChoice(list(range(2,9))),
)

# Ensure that the arrays can take on values in the specified range
parametrization[0].set_bounds(lower=0.01, upper=0.2)
parametrization[1].set_bounds(lower=0.2, upper=0.55)

optimizer = ng.optimizers.PortfolioDiscreteOnePlusOne(parametrization=parametrization, budget=25)
partial_objective_function = partial(objective_function, res_=izh_res, X_stimuli=X, labels=y)

def print_progress(optimizer):
    for i in range(optimizer.budget):
        x = optimizer.ask()
        loss = -partial_objective_function(params=x.value)
        optimizer.tell(x, loss)
        if (i + 1) % 2 == 0:
            print(f"Iteration {i + 1}/{optimizer.budget}, Current loss: {loss}")

print_progress(optimizer)
recommendation = optimizer.provide_recommendation()
optimal_a, optimal_b, optimal_c, optimal_d = recommendation.value

print("Optimal x1:", optimal_a)
print("Optimal x2:", optimal_b)
print("Optimal x3:", optimal_c)
print("Optimal x4:", optimal_d)

c:\Users\singh\miniconda3\Lib\site-packages\nevergrad\parametrization\_datalayers.py:107: NevergradRuntimeWarning: Bounds are 0.19 sigma away from each other at the closest, you should aim for at least 3 for better quality.
  warnings.warn(
c:\Users\singh\miniconda3\Lib\site-packages\nevergrad\parametrization\_datalayers.py:107: NevergradRuntimeWarning: Bounds are 0.35000000000000003 sigma away from each other at the closest, you should aim for at least 3 for better quality.
  warnings.warn(


Iteration 2/25, Current loss: -0.012394326739013195
Iteration 4/25, Current loss: -0.012394162826240063
Iteration 6/25, Current loss: -0.012618731707334518
Iteration 8/25, Current loss: -0.009521330706775188
Iteration 10/25, Current loss: -0.012998191639780998
Iteration 12/25, Current loss: -0.012126754969358444
Iteration 14/25, Current loss: -0.013073457404971123
Iteration 16/25, Current loss: -0.01318083144724369
Iteration 18/25, Current loss: -0.013073457404971123
Iteration 20/25, Current loss: -0.014043139293789864
Iteration 22/25, Current loss: -0.013073457404971123
Iteration 24/25, Current loss: -0.012584158219397068
Optimal x1: 0.010000000000000002
Optimal x2: 0.22235012814720978
Optimal x3: -47
Optimal x4: 2


In [11]:
izh_res.set_exc_parms(a=optimal_a, b=optimal_b, c=optimal_c, d=optimal_d)
opt_spike = izh_res.simulate(X, mem_thr=30, train=False, verbose=True)

100%|██████████| 115/115 [00:25<00:00,  4.51it/s]


In [12]:
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from neucube.sampler import TemporalBinning
from neucube.utils import SeparationIndex

sampler = TemporalBinning(bin_size=10)
opt_state_vec = sampler.sample(opt_spike)
print(opt_state_vec.shape)

num_folds = 10
kf = KFold(n_splits=num_folds)

true_labels = []
predicted_labels = []

for train_index, test_index in tqdm(kf.split(opt_state_vec)):
    X_train_fold, X_test_fold = opt_state_vec[train_index], opt_state_vec[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    svm = SVC(kernel='linear', C=2)  # You can specify different kernels ('linear', 'poly', 'rbf', etc.)
    svm.fit(X_train_fold, y_train_fold)
    y_pred = svm.predict(X_test_fold)
    true_labels.extend(y_test_fold)
    predicted_labels.extend(y_pred)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print("10-Fold Cross-Validation Accuracy:", accuracy)
print(confusion_matrix(true_labels, predicted_labels))
print("separation:", SeparationIndex(opt_state_vec, y))

torch.Size([115, 11000])


10it [00:00, 15.88it/s]

10-Fold Cross-Validation Accuracy: 0.8434782608695652
[[56  8]
 [10 41]]
separation: tensor(0.0140)


In [13]:
izh_res.set_exc_parms(a=0.06, b=0.55, c=-55, d=3)
izh_res.set_inh_parms(a=0.01, b=0.2, c=-65, d=8)
opt_spike = izh_res.simulate(X, mem_thr=30, train=False, verbose=True)

100%|██████████| 115/115 [00:25<00:00,  4.54it/s]


In [14]:
sampler = TemporalBinning(bin_size=10)
opt_state_vec = sampler.sample(opt_spike)
print(opt_state_vec.shape)

num_folds = 10
kf = KFold(n_splits=num_folds)

true_labels = []
predicted_labels = []

for train_index, test_index in tqdm(kf.split(opt_state_vec)):
    X_train_fold, X_test_fold = opt_state_vec[train_index], opt_state_vec[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]

    svm = SVC(kernel='linear', C=2)  # You can specify different kernels ('linear', 'poly', 'rbf', etc.)
    svm.fit(X_train_fold, y_train_fold)
    y_pred = svm.predict(X_test_fold)
    true_labels.extend(y_test_fold)
    predicted_labels.extend(y_pred)

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)
print("10-Fold Cross-Validation Accuracy:", accuracy)
print(confusion_matrix(true_labels, predicted_labels))
print("separation:", SeparationIndex(opt_state_vec, y))

torch.Size([115, 11000])


10it [00:00, 12.12it/s]

10-Fold Cross-Validation Accuracy: 0.8
[[52 12]
 [11 40]]
separation: tensor(0.0112)
